In [14]:
import pandas as pd
import sqlite3

In [15]:
# Path to csv file
CSV_PATH = 'data/heart.csv'

# Path to sqlite databse
SQL_PATH = 'data/MockDatabase.db'

In [16]:
# Read csv
data = pd.read_csv(CSV_PATH)
print(data.shape)
data.head()

(918, 12)


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [17]:
# Code in the following cells was adapted from: https://www.sqlitetutorial.net/sqlite-python/create-tables/
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [18]:
create_table_str = """CREATE TABLE IF NOT EXISTS heart (
                                    id integer PRIMARY KEY,
                                    Age integer,
                                    Sex text,
                                    ChestPainType text,
                                    RestingBP integer,
                                    Cholesterol integer,
                                    FastingBS integer,
                                    RestingECG text,
                                    MaxHR integer,
                                    ExerciseAngina text,
                                    Oldpeak real,
                                    ST_Slope text,
                                    HeartDisease integer
                                );"""

# Create connection
conn = create_connection(SQL_PATH)

# Create table
if conn is not None:
  
    create_table(conn, create_table_str)
    
conn.close()

In [19]:
# Write pandas data to sql table
conn = create_connection(SQL_PATH)
data.to_sql('heart', conn, if_exists='append', index=False)
conn.close()

In [21]:
# Test read
conn = create_connection(SQL_PATH)
data_sql = pd.read_sql("SELECT * FROM heart", conn) # Never use SELECT * in production
conn.close()

print(data_sql.shape)
data_sql.head()

(918, 13)


,id,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,1,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,2,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,3,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,4,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,5,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
